In [ ]:
# Import necessary libraries
from flask import Flask, render_template_string, request
import os
from werkzeug.utils import secure_filename
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
import numpy as np

# Define Flask app
app = Flask(__name__)

# Load the MobileNetV2 pre-trained model for fine-tuned face recognition
model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model = tf.keras.Sequential([
    model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Assuming binary classification (child found or not)
])

# Assuming the model has been trained and saved, load the fine-tuned model
# Uncomment this line if the model is already trained and saved
# model.load_weights('path_to_your_trained_model.h5')

# Dummy image processing and prediction for now
def identify_child(img_path):
    # Load the image and preprocess it for MobileNetV2
    img = image.load_img("C://Users//abhir//OneDrive//Desktop//child1.jpg", target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    # Predict using the model
    prediction = model.predict(img_array)

    # Assuming binary classification with threshold of 0.5
    if prediction[0] > 0.5:
        return "Child Found"
    else:
        return "Child Not Found"

# Route for the homepage with image upload for identification
@app.route('/')
def home():
    return render_template_string('''
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Child Recognition System</title>
        <style>
            * { margin: 0; padding: 0; box-sizing: border-box; }
            body { font-family: Arial, sans-serif; background-color: #f4f4f9; color: #333; }
            header { background-color: #4CAF50; color: white; text-align: center; padding: 16px; }
            nav ul { background-color: #333; overflow: hidden; color: white; padding: 0; text-align: center; margin: 0; }
            nav ul li { display: inline; padding: 20px; }
            nav ul li a { color: white; text-decoration: none; font-size: 18px; }
            .hero { text-align: center; padding: 50px; background-color: #f9f9f9; }
            .hero h2 { color: #4CAF50; font-size: 36px; }
            .hero p { font-size: 18px; margin: 20px 0; }
            button { background-color: #4CAF50; color: white; padding: 10px 20px; border: none; font-size: 18px; cursor: pointer; border-radius: 5px; }
            button:hover { background-color: #45a049; }
            footer { background-color: #333; color: white; text-align: center; padding: 10px; position: fixed; width: 100%; bottom: 0; }
        </style>
    </head>
    <body>
        <header>
            <h1>Welcome to Missing Child Recognition System</h1>
        </header>

        <nav>
            <ul>
                <li><a href="/">Home</a></li>
                <li><a href="/login">Login</a></li>
            </ul>
        </nav>

        <section class="hero">
            <h2>Helping You Find Missing Children</h2>
            <p>Use advanced facial recognition to identify and reunite families.</p>
            <form action="/identify" method="POST" enctype="multipart/form-data">
                <input type="file" name="image" required>
                <button type="submit">Identify Child</button>
            </form>
        </section>

        <footer>
            <p>&copy; 2024 Missing Child Recognition System. All Rights Reserved.</p>
        </footer>
    </body>
    </html>
    ''')

# Route for login page
@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']
        return "Successfully Logged in"  # Add proper authentication logic here
    return render_template_string('''
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Login - Child Recognition System</title>
        <style>
            body { font-family: Arial, sans-serif; background-color: #f4f4f9; color: #333; }
            .login-form { width: 300px; margin: 50px auto; padding: 20px; background-color: #f4f4f9; border-radius: 5px; box-shadow: 0 0 10px rgba(0, 0, 0, 0.1); }
            form { display: flex; flex-direction: column; }
            label { margin-bottom: 5px; font-size: 16px; }
            input { padding: 10px; margin-bottom: 15px; border: 1px solid #ccc; border-radius: 5px; }
            button { background-color: #4CAF50; color: white; padding: 10px; border: none; font-size: 18px; cursor: pointer; border-radius: 5px; }
            button:hover { background-color: #45a049; }
        </style>
    </head>
    <body>
        <div class="login-form">
            <form action="/login" method="POST">
                <label for="username">Username:</label>
                <input type="text" id="username" name="username" required>
                <label for="password">Password:</label>
                <input type="password" id="password" name="password" required>
                <button type="submit">Login</button>
            </form>
        </div>
    </body>
    </html>
    ''')

# Enhanced identification result route
@app.route('/identify', methods=['POST'])
def identify():
    if 'image' not in request.files:
        return "No image uploaded"
    
    file = request.files['image']
    if file.filename == '':
        return "No selected file"

    if file:
        # Save the uploaded image to a secure location
        filename = secure_filename(file.filename)
        filepath = os.path.join('uploads', filename)
        file.save(filepath)

        # Call the MobileNetV2 model for child identification
        result = identify_child(filepath)

        # Display the uploaded image and the result with an enhanced template
        if result == "Child Found":
            message = "Congratulations! The child has been successfully identified."
            color = "#4CAF50"  # Green for success
            icon = "✅"  # Success icon
        else:
            message = "Sorry, no match found. Please try again."
            color = "#FF5722"  # Red for failure
            icon = "❌"  # Failure icon

        return render_template_string(f'''
        <!DOCTYPE html>
        <html lang="en">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>Identification Result</title>
            <style>
                body {{ font-family: Arial, sans-serif; background-color: #f4f4f9; color: #333; text-align: center; padding: 50px; }}
                .result-container {{ border: 2px solid {color}; padding: 20px; display: inline-block; border-radius: 10px; }}
                .result-container h2 {{ color: {color}; }}
                .result-container img {{ max-width: 100%; height: auto; border-radius: 10px; }}
                .result-container p {{ font-size: 18px; }}
                .back-btn {{ background-color: {color}; color: white; padding: 10px 20px; border: none; font-size: 18px; cursor: pointer; border-radius: 5px; }}
                .back-btn:hover {{ background-color: darken({color}, 10%); }}
            </style>
        </head>
        <body>
            <div class="result-container">
                <h2>{icon} Identification Result: {message}</h2>
                <img src="/uploads/{filename}" alt="Uploaded Image">
                <p><strong>Uploaded Image:</strong></p>
                <p>{message}</p>
                <button class="back-btn" onclick="window.location.href='/'">Try Another Image</button>
            </div>
        </body>
        </html>
        ''')

# Run the app in Jupyter Notebook
if __name__ == '__main__':
    app.run(port=5500)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5500
Press CTRL+C to quit
127.0.0.1 - - [28/Oct/2024 10:38:38] "GET / HTTP/1.1" 200 -


1/1 [==============================] - 3s 3s/step


127.0.0.1 - - [28/Oct/2024 10:38:55] "POST /identify HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2024 10:38:55] "GET /uploads/child1.jpg HTTP/1.1" 404 -
127.0.0.1 - - [28/Oct/2024 10:39:04] "GET / HTTP/1.1" 200 -


1/1 [==============================] - 0s 186ms/step


127.0.0.1 - - [28/Oct/2024 10:39:14] "POST /identify HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2024 10:39:14] "GET /uploads/child2.jpeg HTTP/1.1" 404 -
127.0.0.1 - - [28/Oct/2024 11:10:33] "GET / HTTP/1.1" 200 -


1/1 [==============================] - 0s 102ms/step


127.0.0.1 - - [28/Oct/2024 11:10:47] "POST /identify HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2024 11:10:47] "GET /uploads/child1.jpg HTTP/1.1" 404 -
